# Limitations of reference-counted Rc and Arc types in Rust

The important property of `std::rc::Rc` is that clones of it can 
be moved around. Ownership of those values can be moved around 
your program, which can sometimes help to around tricky design
problems. However, it doesn't solve all problems.

## Rc

Consider the code below. Note that we *move* each clone of the
`Rc` into the function being called:

In [2]:
use std::rc::Rc;

fn a(x: Rc<String>) {
    println!("{}", x);
}

fn b(y: Rc<String>) {
    println!("{}", y);
}

pub fn hey() {
    let s = "hey".to_string();
    let r = Rc::new(s);
    a(r.clone());
    b(r.clone());
}

hey()

hey
hey


()

Well the question is: what can you do with that? As far as mutability is 
concerned, not much.  There is support for obtaining a mutable reference
to the thing inside the `Rc`, but the issue is that when you call
`Rc::get_mut()`, what you *actually* get back is an `Option`, which 
will be `None` if you have a cloned `Rc` of your `Rc` anywhere 
else.  So you can't really do very much with this mutation support
other than perhaps some kind of initialization or something.

Example:

The code below is almost the same as the previous one, but here I'm 
trying to obtain a mutable ref to the `String` inside one of the 
clones of the `Rc`:

In [3]:
fn a(mut x: Rc<String>) {
    let mut z = &*Rc::get_mut(&mut x).unwrap();
    println!("{}", z);
}

fn b(y: Rc<String>) {
    println!("{}", y);
}

pub fn hey() {
    let s = "hey".to_string();
    let r = Rc::new(s);
    a(r.clone());
    b(r.clone());
}

hey()

thread '<unnamed>' panicked at 'called `Option::unwrap()` on a `None` value', src/lib.rs:7:39
stack backtrace:
   0: rust_begin_unwind
             at /rustc/9bc8c42bb2f19e745a63f3445f1ac248fb015e53/library/std/src/panicking.rs:493:5
   1: core::panicking::panic_fmt
             at /rustc/9bc8c42bb2f19e745a63f3445f1ac248fb015e53/library/core/src/panicking.rs:92:14
   2: core::panicking::panic
             at /rustc/9bc8c42bb2f19e745a63f3445f1ac248fb015e53/library/core/src/panicking.rs:50:5
   3: run_user_code_3
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.
Segmentation fault.
   0: evcxr::runtime::Runtime::install_crash_handlers::segfault_handler
   1: <unknown>
   2: mi_free
   3: alloc::alloc::dealloc
             at /rustc/9bc8c42bb2f19e745a63f3445f1ac248fb015e53/library/alloc/src/alloc.rs:104:14
      <alloc::alloc::Global as core::alloc::Al

Error: Child process terminated with status: signal: 6 (core dumped)

It doesn't go well. The problem here is that the `Option` you get back from `Rc::get_mut` returns `None` because
you're not allowed to mutate the string since it's being referenced in another `Rc` somewhere else (in function `b()`). So even though `Rc` does easily let you move ownership around (of the wrapper), it doesn't make mutation any easier.

In [45]:
use std::sync::{Arc, Mutex};

#[derive(Debug)]
struct Node {
    id: usize,
    other: Option<Arc<Mutex<Node>>>,
}

let mut n1 = Node { id: 1, other: None };
let mut n2 = Node { id: 2, other: None };

let original = Arc::new(Mutex::new(n2));
let or1 = original.clone();
n1.other = Some(original.clone());

let or2 = {
    let mut n = original.lock().unwrap();
    let tmp = Arc::new(Mutex::new(n1));
    let result = tmp.clone();
    (*n).other = Some(tmp);
    result
};

println!("{:?}", &or1);
println!("{:?}", &or2);
//(or1, or2)

or1.lock().unwrap().id = 3;
println!("{:?}", &or1);
println!("{:?}", &or2);

or2.lock().unwrap().id = 4;
println!("{:?}", &or1);
println!("{:?}", &or2);

if let Some(n) = &or1.lock().unwrap().other {
    n.lock().unwrap().id = 10;
};
println!("{:?}", &or1);
println!("{:?}", &or2);



Mutex { data: Node { id: 2, other: Some(Mutex { data: Node { id: 1, other: Some(Mutex { data: <locked> }) } }) } }
Mutex { data: Node { id: 1, other: Some(Mutex { data: Node { id: 2, other: Some(Mutex { data: <locked> }) } }) } }
Mutex { data: Node { id: 3, other: Some(Mutex { data: Node { id: 1, other: Some(Mutex { data: <locked> }) } }) } }
Mutex { data: Node { id: 1, other: Some(Mutex { data: Node { id: 3, other: Some(Mutex { data: <locked> }) } }) } }
Mutex { data: Node { id: 3, other: Some(Mutex { data: Node { id: 4, other: Some(Mutex { data: <locked> }) } }) } }
Mutex { data: Node { id: 4, other: Some(Mutex { data: Node { id: 3, other: Some(Mutex { data: <locked> }) } }) } }
Mutex { data: Node { id: 3, other: Some(Mutex { data: Node { id: 10, other: Some(Mutex { data: <locked> }) } }) } }
Mutex { data: Node { id: 10, other: Some(Mutex { data: Node { id: 3, other: Some(Mutex { data: <locked> }) } }) } }
